المكتبات

In [ ]:
!pip install feedparser beautifulsoup4 requests html5lib
!pip install sentence-transformers faiss-cpu
!pip install transformers accelerate
!pip install fastapi uvicorn nest_asyncio


In [ ]:
import os
import time
from datetime import datetime
from typing import List, Optional, Dict

import requests
import feedparser
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline


عريف مصادر الأخبار (RSS)

In [ ]:
SOURCES = [
    # id, rss_url
    ("reuters_world", "http://feeds.reuters.com/Reuters/worldNews"),
    ("nyt_world", "https://rss.nytimes.com/services/xml/rss/nyt/World.xml"),
    ("wsj_world", "https://feeds.a.dj.com/rss/RSSWorldNews.xml"),
    ("economist_international", "https://www.economist.com/international/rss.xml"),
    ("cnn_world", "http://rss.cnn.com/rss/cnn_world.rss"),
    ("bbc_world", "http://feeds.bbci.co.uk/news/world/rss.xml"),
    ("dw_world", "https://rss.dw.com/rdf/rss-en-world"),
    ("france24_en", "https://www.france24.com/en/rss"),
    ("guardian_world", "https://www.theguardian.com/world/rss"),
    ("washington_post_world", "http://feeds.washingtonpost.com/rss/world"),
    ("ap_top", "https://apnews.com/apf-topnews?output=rss"),
    ("axios", "https://www.axios.com/feeds/feed.rss"),
    ("the_hill", "https://thehill.com/feed/"),
    ("newsweek", "https://www.newsweek.com/rss"),
    ("alarabiya_en", "https://english.alarabiya.net/.mrss/en.xml"),
    ("cbs_world", "https://www.cbsnews.com/latest/rss/world"),
    ("nbc_world", "http://feeds.nbcnews.com/feeds/worldnews"),
    ("abc_world", "http://feeds.abcnews.com/abcnews/internationalheadlines"),
    ("ft_world", "https://www.ft.com/world/rss"),
]


سكربينق RSS

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36"
}

def fetch_rss(source_id: str, url: str, timeout: int = 10) -> List[Dict]:
    out = []
    try:
        feed = feedparser.parse(url)
        for entry in feed.entries:
            link = entry.get("link", "")
            title = entry.get("title", "").strip()
            summary = entry.get("summary", "").strip()
            published = entry.get("published", "") or entry.get("updated", "")

            out.append({
                "source": source_id,
                "headline": title,
                "summary": summary,
                "link": link,
                "published_raw": published,
            })
    except Exception as e:
        print(f"[RSS] {source_id} error:", e)
    return out

all_rows = []
for sid, url in SOURCES:
    print(">>", sid)
    rows = fetch_rss(sid, url)
    print("  grabbed:", len(rows))
    all_rows.extend(rows)

df_rss = pd.DataFrame(all_rows).drop_duplicates(subset=["source", "headline", "link"])
df_rss.head()


تنظيف التاريخ + سكربينق محتوى المقال

In [ ]:
def parse_date_safe(x: str) -> Optional[datetime]:
    try:
        return pd.to_datetime(x)
    except Exception:
        return None

df_rss["published_at"] = df_rss["published_raw"].apply(parse_date_safe)

def extract_article_text(url: str, timeout: int = 10) -> str:
    """
    سكربينق بسيط للمقال. مو كامل 100% بس يعطيك محتوى كفاية للتحليل.
    """
    if not url:
        return ""

    try:
        resp = requests.get(url, headers=headers, timeout=timeout)
        if resp.status_code >= 400:
            return ""

        html = resp.text
        soup = BeautifulSoup(html, "html5lib")

        # جرب نلقط <article> إذا موجود
        article_tag = soup.find("article")
        if article_tag:
            texts = [p.get_text(" ", strip=True) for p in article_tag.find_all(["p", "h2", "li"])]
            content = " ".join(texts)
            if len(content) > 300:
                return content

        # fallback: divs الشائعة
        candidates = []
        for cls in ["article-body", "story-body", "post-content", "entry-content", "c-article-body"]:
            div = soup.find("div", class_=lambda x: x and cls in x)
            if div:
                candidates.append(div)

        for div in candidates:
            texts = [p.get_text(" ", strip=True) for p in div.find_all(["p", "h2", "li"])]
            content = " ".join(texts)
            if len(content) > 300:
                return content

        # fallback أخير: كل الفقرات
        texts = [p.get_text(" ", strip=True) for p in soup.find_all("p")]
        content = " ".join(texts)
        return content
    except Exception:
        return ""

# سكربينق محتوى المقالات (ممكن تستغرق شوي)
contents = []
for i, row in df_rss.iterrows():
    if i % 50 == 0:
        print("article", i, "/", len(df_rss))
    contents.append(extract_article_text(row["link"]))

df_rss["content"] = contents
df_rss["content"] = df_rss["content"].fillna("").astype(str)

df_rss.to_csv("news_raw_with_content.csv", index=False)
len(df_rss)


Deep Article Scraping

In [ ]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import pandas as pd

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36"
}

def extract_article_text(url: str, timeout: int = 10) -> str:
    """
    المرحلة الثانية من الـ Hybrid Architecture:
    RSS = نقطة دخول رسمية
    Content Scraping = طبقة الإثراء النصي للتحليل الدلالي والمشاعر
    """
    if not url:
        return ""

    try:
        resp = requests.get(url, headers=headers, timeout=timeout)
        if resp.status_code >= 400:
            return ""

        soup = BeautifulSoup(resp.text, "html5lib")

        # محاولة التقاط article tag إن وجد
        article = soup.find("article")
        if article:
            parts = [p.get_text(" ", strip=True) for p in article.find_all(["p", "h2", "li"])]
            text = " ".join(parts)
            if len(text) > 300:
                return text

        # بدائل شائعة لبنية المقال
        candidates = [
            "article-body", "story-body", "entry-content",
            "post-content", "c-article-body", "article__content"
        ]

        for cls in candidates:
            div = soup.find("div", class_=lambda x: x and cls in x)
            if div:
                parts = [p.get_text(" ", strip=True) for p in div.find_all(["p", "h2", "li"])]
                text = " ".join(parts)
                if len(text) > 300:
                    return text


        parts = [p.get_text(" ", strip=True) for p in soup.find_all("p")]
        return " ".join(parts)

    except Exception:
        return ""


def scrape_one(item):
    idx, url = item
    text = extract_article_text(url)
    return idx, text

urls = [(i, row["link"]) for i, row in df_rss.iterrows()]

contents = [""] * len(df_rss)

print("Starting threaded scraping...")
done = 0
total = len(urls)


with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    for idx, text in executor.map(scrape_one, urls):
        contents[idx] = text
        done += 1
        if done % 50 == 0:
            print(f"Processed {done}/{total}")

df_rss["content"] = contents
df_rss["content"] = df_rss["content"].fillna("").astype(str)

df_rss.to_csv("news_raw_with_content.csv", index=False)

print("✅ Done — enriched articles saved.")
len(df_rss)


تصنيف مجال الخبر + الدولة

In [ ]:
df = df_rss.copy()

df["full_text"] = (df["headline"].fillna("") + " " + df["content"].fillna("")).astype(str)

DOMAIN_KEYWORDS = {
    "Security": [
        "war","conflict","military","attack","terror","missile","strike","army",
        "حرب","هجوم","صاروخ","أمن","تفجير","ميليشيا"
    ],
    "Economy": [
        "economy","economic","inflation","interest rate","market","stock","debt",
        "اقتصاد","تضخم","بطالة","سوق","بورصة","نمو","ميزانية"
    ],
    "Energy": [
        "oil","gas","energy","OPEC","fuel","power plant",
        "نفط","غاز","طاقة","برميل","أوبك"
    ],
    "Politics": [
        "president","prime minister","election","government","parliament","sanction",
        "رئيس","انتخابات","حكومة","عقوبات","برلمان","دبلوماسية"
    ],
    "Tech": [
        "technology","AI","cyber","data","software","chip","semiconductor",
        "تقنية","ذكاء اصطناعي","سيبراني","بيانات","خوارزمية"
    ]
}

def classify_domain(text: str) -> str:
    t = text.lower()
    scores = {k: 0 for k in DOMAIN_KEYWORDS}
    for dom, kws in DOMAIN_KEYWORDS.items():
        for kw in kws:
            if kw.lower() in t:
                scores[dom] += 1
    best = max(scores, key=scores.get)
    return best if scores[best] > 0 else "Other"

df["domain"] = df["full_text"].apply(classify_domain)

# خريطة دول بسيطة
COUNTRY_MAP = {
    "Saudi Arabia": ["saudi arabia","riyadh","المملكة العربية السعودية","السعودية"],
    "United States": ["united states","u.s.","usa","washington"],
    "United Kingdom": ["united kingdom","uk","london"],
    "France": ["france","paris"],
    "Germany": ["germany","berlin"],
    "Russia": ["russia","moscow"],
    "China": ["china","beijing"],
    "Iran": ["iran","tehran","إيران"],
    "Israel": ["israel","تل أبيب","إسرائيل"],
}

def extract_country(text: str) -> str:
    low = text.lower()
    for country, kws in COUNTRY_MAP.items():
        for kw in kws:
            if kw.lower() in low:
                return country
    return ""

df["country"] = df["full_text"].apply(extract_country)

df.to_csv("news_enriched_step1.csv", index=False)
df.head()


تحليل مشاعر

In [ ]:
sentiment_model = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=-1
)

def safe_sentiment(text: str):
    text = (text or "").strip()
    if len(text) < 30:
        return ("NEUTRAL", 0.0)
    out = sentiment_model(text[:512])[0]
    return (out["label"], float(out["score"]))

labels, scores = [], []
for i, txt in enumerate(df["content"].tolist()):
    if i % 50 == 0:
        print("sent", i, "/", len(df))
    lab, sc = safe_sentiment(txt)
    labels.append(lab)
    scores.append(sc)

df["sentiment_label"] = labels
df["sentiment_score"] = scores

df.to_csv("news_with_sentiment.csv", index=False)
len(df)


Embeddings + FAISS Index

In [ ]:
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

texts = df["full_text"].fillna("").tolist()
emb = embed_model.encode(
    texts,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True
)

# normalize
emb = emb / (np.linalg.norm(emb, axis=1, keepdims=True) + 1e-10)

dim = emb.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(emb)

faiss.write_index(index, "news_faiss_index.bin")
df.to_parquet("news_final.parquet", index=False)

print("index size:", index.ntotal)


محرك بحث دلالي كفنكشن

In [ ]:
df = pd.read_parquet("news_final.parquet")
index = faiss.read_index("news_faiss_index.bin")

def semantic_search(query: str,
                    top_k: int = 20,
                    domain: Optional[str] = None,
                    country: Optional[str] = None) -> pd.DataFrame:
    if not query.strip():
        return pd.DataFrame()

    q_emb = embed_model.encode([query], convert_to_numpy=True)[0]
    q_emb = q_emb / (np.linalg.norm(q_emb) + 1e-10)

    D, I = index.search(q_emb.reshape(1, -1), top_k * 3)
    scores, idxs = D[0], I[0]

    res = df.iloc[idxs].copy()
    res["semantic_score"] = scores

    if domain:
        res = res[res["domain"] == domain]

    if country:
        res = res[res["country"] == country]

    res = res.sort_values("semantic_score", ascending=False).head(top_k)

    return res[[
        "semantic_score",
        "published_at",
        "source",
        "domain",
        "country",
        "headline",
        "summary",
        "sentiment_label",
        "sentiment_score",
        "link"
    ]]

# تجربة سريعة
semantic_search("Saudi oil production", top_k=5)


Narrative Divergence Engine

In [ ]:
def narrative_divergence(query: str,
                         top_k: int = 200,
                         min_articles_per_country: int = 3):
    """
    يقيس اختلاف السرد بين الدول لنفس الموضوع.
    يرجّع: divergence_index, summary_df
    """
    base_res = semantic_search(query, top_k=top_k)
    if base_res.empty:
        return None, pd.DataFrame()

    # نحتاج كل الأعمدة + embeddings من جديد
    idxs = base_res.index.tolist()
    tmp = df.loc[idxs].copy()
    tmp = tmp[tmp["country"].astype(str) != ""]
    if tmp.empty:
        return None, pd.DataFrame()

    texts = (tmp["headline"].fillna("") + " " + tmp["content"].fillna("")).tolist()
    emb_local = embed_model.encode(texts, convert_to_numpy=True)
    emb_local = emb_local / (np.linalg.norm(emb_local, axis=1, keepdims=True) + 1e-10)

    tmp = tmp.reset_index(drop=True)

    country_centroids = {}
    country_counts = {}
    for country in tmp["country"].unique():
        idx_c = tmp[tmp["country"] == country].index.values
        if len(idx_c) < min_articles_per_country:
            continue
        vecs = emb_local[idx_c]
        c = vecs.mean(axis=0)
        c = c / (np.linalg.norm(c) + 1e-10)
        country_centroids[country] = c
        country_counts[country] = len(idx_c)

    if len(country_centroids) < 2:
        return None, pd.DataFrame()

    countries = list(country_centroids.keys())
    centroids = np.stack([country_centroids[c] for c in countries], axis=0)

    dists = []
    for i in range(len(countries)):
        for j in range(i+1, len(countries)):
            v1, v2 = centroids[i], centroids[j]
            cos_sim = float(np.dot(v1, v2))
            dists.append(1.0 - cos_sim)

    div_index = float(np.mean(dists))

    # summary table
    avg_sents = []
    for c in countries:
        s = tmp[tmp["country"] == c]["sentiment_score"].dropna()
        avg_sents.append(s.mean() if not s.empty else np.nan)

    summary_df = pd.DataFrame({
        "country": countries,
        "article_count": [country_counts[c] for c in countries],
        "avg_sentiment": avg_sents
    }).sort_values("article_count", ascending=False)

    return div_index, summary_df

# تجربة
div_idx, div_table = narrative_divergence("Gaza war", top_k=150)
div_idx, div_table.head()


FastAPI

In [ ]:
## 🔹 Step 12 — Define News Intelligence API (FastAPI)

from fastapi import FastAPI
from pydantic import BaseModel
from typing import Optional

app = FastAPI(
    title="NARRA – News Intelligence API",
    description=(
        "Backend for global news intelligence: semantic search, "
        "sentiment, and narrative divergence between countries."
    ),
    version="1.0.0",
)

# ---------- Request models ----------

class SearchRequest(BaseModel):
    query: str
    top_k: int = 20
    domain: Optional[str] = None
    country: Optional[str] = None


class DivergenceRequest(BaseModel):
    query: str
    top_k: int = 200


# ---------- Health check ----------

@app.get("/health")
def health_check():
    """
    Simple health check to make sure the API is up.
    """
    return {"status": "ok", "service": "news_intelligence_api"}


# ---------- Semantic search endpoint ----------

@app.post("/semantic-search")
def api_semantic_search(body: SearchRequest):
    """
    Semantic search over news articles using FAISS + sentence embeddings.
    """
    res = semantic_search(
        query=body.query,
        top_k=body.top_k,
        domain=body.domain,
        country=body.country,
    )

    # Make sure we always return a JSON-friendly structure
    results = []
    if not res.empty:
        results = res.to_dict(orient="records")

    return {
        "query": body.query,
        "top_k": body.top_k,
        "count": len(results),
        "results": results,
    }


# ---------- Narrative divergence endpoint ----------

@app.post("/narrative-divergence")
def api_narrative_divergence(body: DivergenceRequest):
    """
    Compute narrative divergence index between countries
    for a given topic/query.
    """
    div_idx, table = narrative_divergence(
        query=body.query,
        top_k=body.top_k,
    )

    if div_idx is None or table is None or table.empty:
        return {
            "query": body.query,
            "top_k": body.top_k,
            "divergence_index": None,
            "detail": [],
            "message": "Not enough data to compute divergence (countries/articles too few).",
        }

    detail = table.to_dict(orient="records")

    return {
        "query": body.query,
        "top_k": body.top_k,
        "divergence_index": float(div_idx),
        "detail": detail,
    }


Run the API server inside Colab

In [ ]:
## 🔹 Step 13 — Run FastAPI server inside Colab

import nest_asyncio
import uvicorn

# Allow running uvicorn inside Jupyter/Colab event loop
nest_asyncio.apply()

# Run the server (blocking cell – keep it last)
uvicorn.run(app, host="0.0.0.0", port=8000)
